In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns

from mydatools.features_analyze import get_top_k_corr

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

## Features

In [5]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

**numerical_features**

In [6]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

**categorical_features**

In [7]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [8]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
full_df = pd.get_dummies(full_df, columns=categorical_features, prefix_sep=':')

# add features
new_features = full_df.columns.tolist()[full_df.columns.tolist().index('ds_type') + 1:]
add_features(new_features)

## LightGBM

In [9]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [10]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=50)

[1]	training's auc: 0.702732	valid_1's auc: 0.704609
Training until validation scores don't improve for 50 rounds.
[2]	training's auc: 0.710673	valid_1's auc: 0.716148
[3]	training's auc: 0.71586	valid_1's auc: 0.721955
[4]	training's auc: 0.718406	valid_1's auc: 0.724245
[5]	training's auc: 0.720207	valid_1's auc: 0.725745
[6]	training's auc: 0.721031	valid_1's auc: 0.726873
[7]	training's auc: 0.721282	valid_1's auc: 0.72692
[8]	training's auc: 0.721362	valid_1's auc: 0.726695
[9]	training's auc: 0.723952	valid_1's auc: 0.728975
[10]	training's auc: 0.724079	valid_1's auc: 0.72884
[11]	training's auc: 0.724511	valid_1's auc: 0.729234
[12]	training's auc: 0.724876	valid_1's auc: 0.729284
[13]	training's auc: 0.72677	valid_1's auc: 0.730961
[14]	training's auc: 0.726956	valid_1's auc: 0.731219
[15]	training's auc: 0.727442	valid_1's auc: 0.731895
[16]	training's auc: 0.727672	valid_1's auc: 0.73212
[17]	training's auc: 0.727823	valid_1's auc: 0.732214
[18]	training's auc: 0.728105	vali

[153]	training's auc: 0.767151	valid_1's auc: 0.7607
[154]	training's auc: 0.767255	valid_1's auc: 0.760762
[155]	training's auc: 0.767269	valid_1's auc: 0.760771
[156]	training's auc: 0.767398	valid_1's auc: 0.760887
[157]	training's auc: 0.767517	valid_1's auc: 0.760914
[158]	training's auc: 0.767625	valid_1's auc: 0.76094
[159]	training's auc: 0.767728	valid_1's auc: 0.760968
[160]	training's auc: 0.767838	valid_1's auc: 0.76096
[161]	training's auc: 0.767932	valid_1's auc: 0.760929
[162]	training's auc: 0.767996	valid_1's auc: 0.760951
[163]	training's auc: 0.768077	valid_1's auc: 0.76104
[164]	training's auc: 0.768141	valid_1's auc: 0.761066
[165]	training's auc: 0.768297	valid_1's auc: 0.761038
[166]	training's auc: 0.768379	valid_1's auc: 0.761046
[167]	training's auc: 0.768479	valid_1's auc: 0.761053
[168]	training's auc: 0.768609	valid_1's auc: 0.761115
[169]	training's auc: 0.768646	valid_1's auc: 0.761118
[170]	training's auc: 0.768746	valid_1's auc: 0.761174
[171]	training'

In [11]:
# bst = lgb.train(params, trn_lgb, 1000, valid_sets=[trn_lgb, val_lgb], early_stopping_rounds=20,
#                 init_model=bst, 
#                 learning_rates=lambda iter: 0.1 * (0.99 ** iter))

In [12]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_3,364
EXT_SOURCE_1,288
EXT_SOURCE_2,277
DAYS_BIRTH,207
AMT_CREDIT,187
AMT_ANNUITY,160
AMT_GOODS_PRICE,142
DAYS_EMPLOYED,134
DAYS_ID_PUBLISH,111
OWN_CAR_AGE,93


## predict

In [13]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)